# INSIGHTS SQL 

## 1. Demanda de uso de bicicletas por año

In [0]:
%sql
SELECT
  YEAR(date) AS `año`,
  SUM(sum_total) AS total
FROM tfm.fact_bike_agg
WHERE YEAR(date) <> 2019
GROUP BY ALL
ORDER BY YEAR(date) ASC

año,total
2020,19562792
2021,27129753
2022,29838442
2023,35107120
2024,44302799


## 2. Promedio demanda usuarios bicicletas por mes y variación % respecto al mes anterior

In [0]:
%sql
WITH fact_bike_month AS (
  SELECT
    n_mes,
    mes,
    ROUND(AVG(total),0) AS avg_total
  FROM (
    SELECT
      YEAR(date) AS `año`,
      MONTH(date) AS n_mes,
      CASE
        WHEN MONTH(date) = 1 THEN 'enero'
        WHEN MONTH(date) = 2 THEN 'febrero'
        WHEN MONTH(date) = 3 THEN 'marzo'
        WHEN MONTH(date) = 4 THEN 'abril'
        WHEN MONTH(date) = 5 THEN 'mayo'
        WHEN MONTH(date) = 6 THEN 'junio'
        WHEN MONTH(date) = 7 THEN 'julio'
        WHEN MONTH(date) = 8 THEN 'agosto'
        WHEN MONTH(date) = 9 THEN 'septiembre'
        WHEN MONTH(date) = 10 THEN 'octubre'
        WHEN MONTH(date) = 11 THEN 'noviembre'
        WHEN MONTH(date) = 12 THEN 'diciembre'
      END as mes, 
      SUM(sum_total) AS total
    FROM tfm.fact_bike_agg
    WHERE YEAR(date) <> 2019
    GROUP BY ALL
  )
  GROUP BY ALL
)

SELECT
  n_mes,
  mes,
  avg_total,
  ROUND(((avg_total/LAG(avg_total) OVER (ORDER BY n_mes)) - 1) * 100, 2) `AS MoM(%)`
FROM fact_bike_month

n_mes,mes,avg_total,AS MoM(%)
1,enero,1409731.0,null
2,febrero,1360258.0,-3.51
3,marzo,1840047.0,35.27
4,abril,2187696.0,18.89
5,mayo,2923291.0,33.62
6,junio,3316183.0,13.44
7,julio,3382877.0,2.01
8,agosto,3498528.0,3.42
9,septiembre,3519218.0,0.59
10,octubre,3415479.0,-2.95


## 3. Demanda usuarios de metro: días laborables vs fines de semana y festivos

In [0]:
%sql
WITH fact_metro_calendar AS (
  SELECT
  CASE
    WHEN (calendar.is_holiday = 1) OR (calendar.is_weekend = 1) THEN 'festivo/fin de semana'
    ELSE 'laborable'
  END AS day_type,
  fact.day,
  SUM(fact.total_ridership) AS total_ridership
  FROM tfm.fact_metro fact
  LEFT JOIN tfm.dim_calendar calendar ON fact.day = calendar.date
  GROUP BY ALL
)

SELECT
  day_type,
  ROUND(AVG(total_ridership), 0) AS avg_total
FROM fact_metro_calendar
GROUP BY ALL

day_type,avg_total
laborable,3022384.0
festivo/fin de semana,1760577.0


## 4. Impacto de las lluvias/nieve en la demanda de usuarios de bicicleta

In [0]:
%sql
WITH fact_bike_weather AS (
  SELECT 
    fact.date,
    fact.sum_total,
    CASE
      WHEN (weather.prcp > 0) OR (weather.snow > 0) THEN 'rain/snow'
      ELSE 'clear'
    END AS weather
  FROM tfm.fact_bike_agg fact
  LEFT JOIN (SELECT * FROM tfm.fact_weather WHERE weather_station_id = 'USW00094728') weather ON fact.date = weather.date
)

SELECT
  weather,
  ROUND(AVG(sum_total), 0) AS avg_total
FROM fact_bike_weather
GROUP BY ALL

weather,avg_total
clear,81675.0
rain/snow,70925.0


## 5. Impacto de la temperatura en la demanda de uso de bicicletas

In [0]:
%sql
WITH fact_bike_weather AS (
  SELECT
    fact.date,
    fact.sum_total,
    weather.trange
  FROM tfm.fact_bike_agg fact
  LEFT JOIN (
    SELECT
      date,
      CASE
        WHEN tmax <= 5 THEN '< 5°C' 
        WHEN tmax <= 10 THEN '5°C - 10°C'
        WHEN tmax <= 15 THEN '10°C - 15°C'
        WHEN tmax <= 20 THEN '15°C - 20°C'
        WHEN tmax <= 25 THEN '20°C - 25°C'
        WHEN tmax <= 30 THEN '25°C - 30°C'
        ELSE '> 30°C'
      END trange
    FROM tfm.fact_weather
    WHERE weather_station_id = 'USW00094728'
  ) weather
  ON fact.date = weather.date
)

SELECT
  trange,
  ROUND(AVG(sum_total),0) AS avg_total
FROM fact_bike_weather
GROUP BY ALL
HAVING trange IS NOT NULL
ORDER BY AVG(sum_total) DESC

trange,avg_total
25°C - 30°C,116199.0
> 30°C,110601.0
20°C - 25°C,109345.0
15°C - 20°C,85068.0
10°C - 15°C,67089.0
5°C - 10°C,54478.0
< 5°C,40612.0


## 6. Demanda de usuarios de autobús por día de la semana

In [0]:
%sql
SELECT
  day_of_week,
  ROUND(AVG(total), 0) AS avg_total
FROM(
      SELECT
        day,
        CASE
          WHEN DAYOFWEEK(day) = 1 THEN 'domingo'
          WHEN DAYOFWEEK(day) = 2 THEN 'lunes'
          WHEN DAYOFWEEK(day) = 3 THEN 'martes'
          WHEN DAYOFWEEK(day) = 4 THEN 'miércoles'
          WHEN DAYOFWEEK(day) = 5 THEN 'jueves'
          WHEN DAYOFWEEK(day) = 6 THEN 'viernes'
          WHEN DAYOFWEEK(day) = 7 THEN 'sábado'
        END as day_of_week,
        SUM(total_ridership) AS total
      FROM tfm.fact_bus
      GROUP BY ALL
)
GROUP BY ALL

day_of_week,avg_total
jueves,1121502.0
viernes,1076245.0
miércoles,1146405.0
lunes,1044335.0
martes,1133167.0
domingo,533559.0
sábado,684099.0


## 7. Demanda días festivos vs média diaria metro

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW table_valor AS
SELECT
  ROUND(AVG(total), 0) AS valor
FROM (
  SELECT 
    day AS date,
    SUM(total_ridership) AS total
  FROM tfm.fact_metro
  GROUP BY ALL
);

WITH fact_metro_holiday AS (
  SELECT
    fact.day AS date,
    holiday.holiday_name,
    SUM(fact.total_ridership) AS total    
  FROM tfm.fact_metro fact
  LEFT JOIN tfm.dim_holiday holiday ON fact.day = holiday.date
  GROUP BY ALL
)

SELECT 
  holiday_name,
  ROUND(AVG(total), 0) AS avg_total,
  ((AVG(total) / (SELECT valor FROM table_valor)) - 1) * 100 AS `holiday_vs_avgday(%)`
FROM fact_metro_holiday
WHERE holiday_name IS NOT NULL
GROUP BY ALL
ORDER BY AVG(total) DESC

holiday_name,avg_total,holiday_vs_avgday(%)
Election Day,3698677.0,40.702799389511846
Lincoln’s Birthday,3427053.0,30.369845962819863
Veterans Day,2703453.0,2.843114280888459
Columbus Day,2434531.0,-7.3870665374274
Presidents’ Day,2187039.0,-16.80200523753802
Martin Luther King Jr. Day,2096040.0,-20.263740929031513
Juneteenth,2049896.0,-22.01912264390675
Labor Day,1614548.0,-38.58035634127079
Independence Day,1508477.0,-42.61544419404759
New Year’s Day,1458520.0,-44.51587771368227


## 8. Analisis temporal de preciptaciones en la estación meteorológica Central Park

In [0]:
%sql
SELECT
  YEAR(date) AS `año`,
  MONTH(date) AS n_mes,
  CASE
    WHEN MONTH(date) = 1 THEN 'enero'
    WHEN MONTH(date) = 2 THEN 'febrero'
    WHEN MONTH(date) = 3 THEN 'marzo'
    WHEN MONTH(date) = 4 THEN 'abril'
    WHEN MONTH(date) = 5 THEN 'mayo'
    WHEN MONTH(date) = 6 THEN 'junio'
    WHEN MONTH(date) = 7 THEN 'julio'
    WHEN MONTH(date) = 8 THEN 'agosto'
    WHEN MONTH(date) = 9 THEN 'septiembre'
    WHEN MONTH(date) = 10 THEN 'octubre'
    WHEN MONTH(date) = 11 THEN 'noviembre'
    WHEN MONTH(date) = 12 THEN 'diciembre'
  END as mes,
  ROUND(SUM(prcp),1) AS prcp
FROM tfm.fact_weather
WHERE weather_station_id = 'USW00094728'
GROUP BY ALL
ORDER BY YEAR(date), MONTH(date)

año,n_mes,mes,prcp
2020,1,enero,49.1
2020,2,febrero,64.7
2020,3,marzo,96.1
2020,4,abril,114.2
2020,5,mayo,42.0
2020,6,junio,44.7
2020,7,julio,167.1
2020,8,agosto,127.9
2020,9,septiembre,100.1
2020,10,octubre,128.3


## 9. Promedio distancia bicicletas por mes en el año 2024

In [0]:
%sql
SELECT
  MONTH(date) AS mes,
  ROUND(AVG(avg_distancia),2) AS avg_distancia
FROM tfm.fact_bike_agg
WHERE YEAR(date) = 2024
GROUP BY ALL
ORDER BY MONTH(date)

mes,avg_distancia
1,1.96
2,2.04
3,2.14
4,2.25
5,2.39
6,2.54
7,2.59
8,2.48
9,2.54
10,2.49


## 10. Estación de bicicletas más utilizada por año

In [0]:
%sql
WITH 

station24 AS (
  SELECT
    2024 AS `año`,
    dim.bike_station_name,
    SUM(fact.sum_total) AS total
  FROM tfm.fact_bike_agg_station fact
  LEFT JOIN tfm.dim_bike AS dim ON fact.bike_start_station_id = dim.bike_station_id
  WHERE dim.bike_station_name IS NOT NULL AND YEAR(date) = 2024
  GROUP BY ALL
  ORDER BY SUM(fact.sum_total) DESC
  LIMIT 1
),

station23 AS (
  SELECT
    2023 AS `año`,
    dim.bike_station_name,
    SUM(fact.sum_total) AS total
  FROM tfm.fact_bike_agg_station fact
  LEFT JOIN tfm.dim_bike AS dim ON fact.bike_start_station_id = dim.bike_station_id
  WHERE dim.bike_station_name IS NOT NULL AND YEAR(date) = 2023
  GROUP BY ALL
  ORDER BY SUM(fact.sum_total) DESC
  LIMIT 1
),

station22 AS (
  SELECT
    2022 AS `año`,
    dim.bike_station_name,
    SUM(fact.sum_total) AS total
  FROM tfm.fact_bike_agg_station fact
  LEFT JOIN tfm.dim_bike AS dim ON fact.bike_start_station_id = dim.bike_station_id
  WHERE dim.bike_station_name IS NOT NULL AND YEAR(date) = 2022
  GROUP BY ALL
  ORDER BY SUM(fact.sum_total) DESC
  LIMIT 1
),

station21 AS (
  SELECT
    2021 AS `año`,
    dim.bike_station_name,
    SUM(fact.sum_total) AS total
  FROM tfm.fact_bike_agg_station fact
  LEFT JOIN tfm.dim_bike AS dim ON fact.bike_start_station_id = dim.bike_station_id
  WHERE dim.bike_station_name IS NOT NULL AND YEAR(date) = 2021
  GROUP BY ALL
  ORDER BY SUM(fact.sum_total) DESC
  LIMIT 1
),

station20 AS (
  SELECT
    2020 AS `año`,
    dim.bike_station_name,
    SUM(fact.sum_total) AS total
  FROM tfm.fact_bike_agg_station fact
  LEFT JOIN tfm.dim_bike AS dim ON fact.bike_start_station_id = dim.bike_station_id
  WHERE dim.bike_station_name IS NOT NULL AND YEAR(date) = 2020
  GROUP BY ALL
  ORDER BY SUM(fact.sum_total) DESC
  LIMIT 1
)
--------------------------------------------------------------------------------

SELECT
  `año`,
  bike_station_name,
  total
FROM station24

UNION ALL 

SELECT
  `año`,
  bike_station_name,
  total
FROM station23

UNION ALL

SELECT
  `año`,
  bike_station_name,
  total
FROM station22

UNION ALL

SELECT
  `año`,
  bike_station_name,
  total
FROM station21

UNION ALL

SELECT
  `año`,
  bike_station_name,
  total
FROM station20


año,bike_station_name,total
2024,W 21 St & 6 Ave,163490
2023,W 21 St & 6 Ave,139935
2022,W 21 St & 6 Ave,129016
2021,W 21 St & 6 Ave,127096
2020,1 Ave & E 68 St,101013


## 11. Top 10 rutas más recurrentes en bicicleta en el año 2024

In [0]:
%sql
SELECT 
  bike_start_station_id,
  bike_end_station_id,
  SUM(total) AS total
FROM tfm.fact_bike
WHERE YEAR(started_date) = 2024
GROUP BY bike_start_station_id, bike_end_station_id
HAVING (bike_start_station_id IS NOT NULL) AND (bike_end_station_id IS NOT NULL)
ORDER BY total DESC
LIMIT 10

bike_start_station_id,bike_end_station_id,total
6876.04,6876.04,15571
6912.01,6912.01,12616
6599.01,6599.01,8827
5368.03,5297.02,7367
4590.01,4590.01,6232
6140.05,6266.06,6089
6599.01,6814.01,5711
7141.07,7141.07,5668
7323.09,7323.09,5636
5374.01,5294.04,5631
